In [1]:
%pip install ultralytics opencv-python matplotlib numpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [2]:
model = YOLO("yolov8n-seg.pt")   # you can switch to yolov8s-seg.pt later


In [7]:
model.train(
    data="cracks/data.yaml",
    epochs=50,
    imgsz=640,
    batch=8,
    patience=20,
    name="surface_defect_seg"
)



Ultralytics 8.3.240  Python-3.12.4 torch-2.5.1+cpu CPU (Intel Core i5-6300U 2.40GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=cracks/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=surface_defect_seg4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=True, p

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000183D5C96450>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    

In [10]:
trained_model = YOLO(r"C:\Users\Gowri Shankar\Downloads\Kambaa\Task 2\runs\segment\surface_defect_seg4\weights\best.pt")


In [23]:
image_path = r"C:\Users\Gowri Shankar\Downloads\Kambaa\Task 2\cracks\train\images\-2023-01-16-5-50-44_png.rf.1a97258540b89f7106fbc9ee8fc58c65.jpg"  # put your image path here
results = trained_model(image_path)



image 1/1 C:\Users\Gowri Shankar\Downloads\Kambaa\Task 2\cracks\train\images\-2023-01-16-5-50-44_png.rf.1a97258540b89f7106fbc9ee8fc58c65.jpg: 640x640 2 cracks, 1553.9ms
Speed: 100.4ms preprocess, 1553.9ms inference, 47.1ms postprocess per image at shape (1, 3, 640, 640)


In [24]:
annotated_img = results[0].plot()
plt.imshow(annotated_img)
plt.axis("off")


(np.float64(-0.5), np.float64(311.5), np.float64(311.5), np.float64(-0.5))

In [25]:
masks = results[0].masks.data.cpu().numpy()    # (N, H, W)
classes = results[0].boxes.cls.cpu().numpy()   # class IDs


In [35]:
img = cv2.imread(image_path)
height, width, _ = img.shape
total_pixels = height * width


In [36]:
class_names = {0: "Crack", 1: "Efflorescence", 2: "Algae/Fungus"}
defect_areas = {}

for i, cls in enumerate(classes):
    cls = int(cls)
    area_pixels = np.sum(masks[i])
    defect_areas[cls] = defect_areas.get(cls, 0) + area_pixels

defect_areas


{1: np.uint64(2410)}

In [37]:
total_defect_pixels = sum(defect_areas.values())
severity_percent = (total_defect_pixels / total_pixels) * 100

severity_percent


np.float64(2.4757560815253123)

In [38]:
def get_severity(p):
    if p <= 5:
        return "0-5%"
    elif p <= 10:
        return "6-10%"
    elif p <= 25:
        return "11-25%"
    else:
        return ">25%"

severity_level = get_severity(severity_percent)
severity_level


'0-5%'

In [39]:
product_rules = {
    "Algae/Fungus": {
        "0-5%": ["Damp Protect Interior", "WP Putty"],
        "6-10%": ["Damp Protect Interior", "Crystal Seal"],
        "11-25%": ["Damp Lock", "Crystal Seal"],
        ">25%": ["Super 2K on plaster", "Damp Protect Interior"]
    },
    "Efflorescence": {
        ">25%": ["Super 2K on brick wall", "Crystal Seal"]
    }
}


In [40]:
recommended_products = set()

for cls_id in defect_areas.keys():
    defect_name = class_names[cls_id]
    if defect_name in product_rules:
        products = product_rules[defect_name].get(severity_level, [])
        recommended_products.update(products)

recommended_products


set()

In [41]:
scale_factor = 0.0001  # adjust based on real-world calibration
wall_area_sqft = total_pixels * scale_factor

coverage_per_litre = 120  # sq.ft per litre
paint_required_litres = wall_area_sqft / coverage_per_litre

paint_required_litres


0.08112000000000001

In [42]:
print("Detected Issues:")
for cls_id in defect_areas:
    print("-", class_names[cls_id])

print("\nSeverity Level:", severity_level)
print("\nRecommended Products:")
for p in recommended_products:
    print("-", p)

print(f"\nEstimated Paint Required: {paint_required_litres:.2f} Litres")


Detected Issues:
- Efflorescence

Severity Level: 0-5%

Recommended Products:

Estimated Paint Required: 0.08 Litres
